# Corpus compilation
- text type and genres
- characteristics according to task

In [2]:
# Requests in python
import urllib3
# Regular expression library
import re
# Transform html into a tree im memory
from bs4 import BeautifulSoup
import os
# Get just the text in a html document
import justext

In [15]:
# Disable https warning
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)
# Pass as a browser 
user_agent = {'user-agent': 'Mozilla/5.0 (Windows NT 6.1; Win64; x64)'}

# Pool of boots to make requests
http = urllib3.PoolManager(10, headers=user_agent)


class Crawler:
    
    def __init__(self, corpus_path, max_files, seed_url, url_pattern):
        self.corpus_path = corpus_path # corpus address
        self.max_files = max_files # max amount of sizes
        self.seed_url = seed_url # root url
        self.url_pattern = url_pattern # Select links of interest
        self.visited_links = {} # Hash to store viseted links
        self.to_be_visited = [] # List of link
        
        # If path not exists create it
        if not os.path.exists(self.corpus_path):
            os.makedirs(self.corpus_path)
        
    def crawl(self):
        first_urls = self.get_page(self.seed_url)
        self.add_links(first_urls)
        next_link = self.get_next_link()
        
        file_counter = 1
        while next_link and file_counter < self.max_files:
            links = self.get_page(next_link)
            self.add_links(links)
            next_link = self.get_next_link()
            file_counter += 1
        
    
    def get_page(self, url):
        print("getting page {}".format(url))
        response = http.request('GET', url)

        # store text content
        paragraphs = justext.justext(response.data, justext.get_stoplist("Portuguese"))
        with open("{}/{}.txt".format(self.corpus_path, url.replace(".", "_").replace("/","-")[8:]), "w") as output_file:
            for paragraph in paragraphs:
                # Boilerplate is everthing that is not the main text
                if not paragraph.is_boilerplate:
                    output_file.write(paragraph.text)
        
        # get links
        soup = BeautifulSoup(response.data, 'html.parser')  
        
        links = [link.get('href') for link in soup.findAll('a', attrs={'href': re.compile(self.url_pattern)})]
        return links

    def add_links(self, links):
        links = list(set(links))
        self.to_be_visited.extend([link for link in links if link not in self.visited_links])

    def get_next_link(self):
        next_link = self.to_be_visited.pop(0)
        self.visited_links[next_link] = None
        return next_link

In [18]:
crawler_seguranca = Crawler("data/corpora/sensacionalista", 50, 
                             "https://www.sensacionalista.com.br/2019/04/09/prefeito-crivella-e-substituido-por-um-cone-da-cet-rio-e-ninguem-nota/", 
                            "^https://www\.sensacionalista\.com\.br/\d+")

crawler_educacao = Crawler("data/corpora/politica", 500,
                          "https://www1.folha.uol.com.br/poder/2019/04/bolsonaro-se-recusa-a-enquadrar-filho-que-reabre-crise-com-mourao.shtml",
                          "^https://www1\.folha\.uol\.com\.br/poder/\d+")

In [19]:
crawler_seguranca.crawl()
crawler_educacao.crawl()

getting page https://www.sensacionalista.com.br/2019/04/09/prefeito-crivella-e-substituido-por-um-cone-da-cet-rio-e-ninguem-nota/
getting page https://www.sensacionalista.com.br/2019/04/08/alunos-terao-que-entoar-cancao-da-despedida-para-velez-em-todas-as-escolas/
getting page https://www.sensacionalista.com.br/2019/04/15/edir-macedo-ganha-passaporte-diplomatico-e-ja-pode-ir-para-o-inferno/
getting page https://www.sensacionalista.com.br/2019/04/07/bolsonaro-tem-30-de-ruim-ou-pessimo-e-70-fugiram-so-de-ouvir-seu-nome/
getting page https://www.sensacionalista.com.br/2019/04/15/petista-morre-engasgado-ao-saber-que-foi-lula-que-deu-passaporte-a-macedo/
getting page https://www.sensacionalista.com.br/2019/04/09/prefeito-crivella-e-substituido-por-um-cone-da-cet-rio-e-ninguem-nota/
getting page https://www.sensacionalista.com.br/2019/04/08/jet-ski-por-aplicativo-faz-sucesso-e-desbanca-patinete-no-rio/
getting page https://www.sensacionalista.com.br/2019/04/09/crivella-pede-que-cariocas-nao-

getting page https://www1.folha.uol.com.br/poder/2019/03/preso-ha-1-ano-lula-tem-rotina-com-advogados-tv-e-ate-videos-de-reunioes-do-pt.shtml
getting page https://www1.folha.uol.com.br/poder/2019/01/na-onda-bolsonarista-otimismo-com-proximo-congresso-e-o-mais-alto-dos-ultimos-anos.shtml
getting page https://www1.folha.uol.com.br/poder/2019/04/alexandre-de-moraes-relata-e-trava-caso-sobre-ele-mesmo-no-supremo.shtml
getting page https://www1.folha.uol.com.br/poder/2018/04/lula-e-preso.shtml
getting page https://www1.folha.uol.com.br/poder/2019/04/bolsonaro-se-recusa-a-enquadrar-filho-que-reabre-crise-com-mourao.shtml
getting page https://www1.folha.uol.com.br/poder/2019/04/saiba-quem-sao-os-ministros-responsaveis-pelo-julgamento-de-lula-no-stj.shtml
getting page https://www1.folha.uol.com.br/poder/2019/04/stj-julga-nesta-terca-recurso-de-lula-contra-condenacao-do-triplex.shtml
getting page https://www1.folha.uol.com.br/poder/2019/04/caixa-2-ficou-impraticavel-por-causa-de-impostos-afirma

getting page https://www1.folha.uol.com.br/poder/2018/12/cansei-de-levar-bola-nas-costas-diz-moro-sobre-troca-de-judiciario-por-executivo.shtml
getting page https://www1.folha.uol.com.br/poder/2019/02/bolsonaro-manda-pf-investigar-laranjas-e-diz-que-bebianno-pode-voltar-as-origens.shtml
getting page https://www1.folha.uol.com.br/poder/2019/03/choques-entre-nucleos-pragmatico-e-ideologico-atrapalham-governo-bolsonaro.shtml
getting page https://www1.folha.uol.com.br/poder/2019/04/deputada-ameacada-apos-denunciar-laranjal-do-psl-relata-abandono-na-infancia.shtml
getting page https://www1.folha.uol.com.br/poder/2019/04/sheherazade-ve-factoide-em-acao-para-derrubar-mourao-apos-vice-curtir-um-post-dela.shtml
getting page https://www1.folha.uol.com.br/poder/2019/04/feliciano-rebate-sheherazade-e-defende-afastar-mourao-em-nome-de-estabilidade.shtml
getting page https://www1.folha.uol.com.br/poder/2019/04/stf-deve-aceitar-pedido-e-adiar-julgamento-sobre-prisao-apos-2a-instancia.shtml
getting pa

getting page https://www1.folha.uol.com.br/poder/2019/03/sob-criticas-da-lava-jato-toffoli-diz-que-nao-sao-herois-que-resolvem-os-problemas.shtml
getting page https://www1.folha.uol.com.br/poder/2017/03/1862602-anulada-operacao-castelo-de-areia-antecipou-caracteristicas-da-lava-jato.shtml
getting page https://www1.folha.uol.com.br/poder/2018/10/justica-uruguaia-nega-extradicao-e-determina-liberdade-de-doleiros-presos-na-cambio-desligo.shtml
getting page https://www1.folha.uol.com.br/poder/2019/03/sem-comentar-inquerito-toffoli-diz-que-sociedade-inteira-e-vitima-de-fake-news.shtml
getting page https://www1.folha.uol.com.br/poder/2019/03/oas-mantinha-esquema-de-caixa-3-em-campanhas-segundo-delatores.shtml
getting page https://www1.folha.uol.com.br/poder/2017/04/1875621-assista-aos-videos-das-principais-delacoes-de-executivos-da-odebrecht.shtml
getting page https://www1.folha.uol.com.br/poder/2019/03/fachin-pede-prioridade-a-toffoli-para-julgar-sigilo-de-whatsapp-e-acordo-da-jbs.shtml
get

getting page https://www1.folha.uol.com.br/poder/2018/12/juiz-arquiva-inquerito-que-investigava-feliciano-por-estupro.shtml
getting page https://www1.folha.uol.com.br/poder/2019/04/se-fosse-presidente-escolheria-outras-pessoas-para-trabalhar-comigo-diz-mourao.shtml
getting page https://www1.folha.uol.com.br/poder/2019/03/contundente-janaina-chega-a-assembleia-descolada-de-doria-e-ja-rejeita-a-prefeitura.shtml
getting page https://www1.folha.uol.com.br/poder/2019/04/para-maioria-da-populacao-golpe-de-1964-deveria-ser-desprezado-diz-datafolha.shtml
getting page https://www1.folha.uol.com.br/poder/2019/04/alckmin-e-ciro-criticam-improviso-e-vassalagem-do-brasil-aos-eua-sob-bolsonaro.shtml
getting page https://www1.folha.uol.com.br/poder/2018/11/ministro-do-stj-nega-novo-recurso-de-lula-para-reverter-condenacao.shtml
getting page https://www1.folha.uol.com.br/poder/2019/04/presidente-do-stj-ve-lula-como-cidadao-comum-e-nega-falha-em-julgamento.shtml
getting page https://www1.folha.uol.com.

getting page https://www1.folha.uol.com.br/poder/2018/11/bolsonaro-anuncia-moro-no-superministerio-da-justica-e-promete-nao-interferir-no-combate-a-corrupcao.shtml
getting page https://www1.folha.uol.com.br/poder/2019/03/em-dois-meses-moro-coleciona-recuos-apos-ordens-de-bolsonaro.shtml
getting page https://www1.folha.uol.com.br/poder/2018/11/acompanhe-a-formacao-dos-ministerios-de-bolsonaro.shtml
getting page https://www1.folha.uol.com.br/poder/2019/04/presidente-do-stj-ve-lula-como-cidadao-comum-e-nega-falha-em-julgamento.shtml
getting page https://www1.folha.uol.com.br/poder/2019/03/maia-diz-que-projeto-de-moro-e-copia-e-cola-e-que-ministro-confunde-as-bolas.shtml
getting page https://www1.folha.uol.com.br/poder/2019/02/entenda-as-evidencias-e-as-versoes-dos-envolvidos-em-esquema-de-laranjas-do-psl.shtml
getting page https://www1.folha.uol.com.br/poder/2019/02/bolsonaro-assina-pacote-anticrime-de-moro-e-medidas-irao-ao-congresso-nesta-terca-feira.shtml
getting page https://www1.folh

getting page https://www1.folha.uol.com.br/poder/2019/02/entenda-as-evidencias-e-as-versoes-dos-envolvidos-em-esquema-de-laranjas-do-psl.shtml
getting page https://www1.folha.uol.com.br/poder/2019/02/bolsonaro-assina-pacote-anticrime-de-moro-e-medidas-irao-ao-congresso-nesta-terca-feira.shtml
getting page https://www1.folha.uol.com.br/poder/2019/04/ex-chefe-da-pf-muda-de-lado-e-orienta-grandes-empresas-em-acoes-anticorrupcao.shtml
getting page https://www1.folha.uol.com.br/poder/2019/04/moro-e-aprovado-por-59-e-titular-do-turismo-tem-11-diz-datafolha.shtml
getting page https://www1.folha.uol.com.br/poder/2019/02/entenda-os-principais-pontos-do-pacote-anticrime-de-moro.shtml
getting page https://www1.folha.uol.com.br/poder/2019/04/se-der-errado-a-culpa-e-de-bolsonaro-brinca-moro-sobre-perfil-no-twitter.shtml
getting page https://www1.folha.uol.com.br/poder/2018/12/cansei-de-levar-bola-nas-costas-diz-moro-sobre-troca-de-judiciario-por-executivo.shtml
getting page https://www1.folha.uol.c

getting page https://www1.folha.uol.com.br/poder/2019/03/entenda-a-sequencia-de-tres-derrotas-da-lava-jato-em-uma-semana-no-supremo.shtml
getting page https://www1.folha.uol.com.br/poder/2019/03/moro-lamenta-congelamento-de-pacote-anticrime-na-camara.shtml
getting page https://www1.folha.uol.com.br/poder/2019/02/entenda-os-principais-pontos-do-pacote-anticrime-de-moro.shtml
getting page https://www1.folha.uol.com.br/poder/2019/03/irritado-com-criticas-de-bolsonaro-maia-ironiza-interferencia-no-legislativo.shtml
getting page https://www1.folha.uol.com.br/poder/2019/02/bebianno-e-demitido-e-caso-dos-laranjas-do-psl-leva-a-primeira-queda-de-ministro-do-governo-bolsonaro.shtml
getting page https://www1.folha.uol.com.br/poder/2019/02/queda-de-ministro-machuca-todo-mundo-mas-e-pagina-virada-diz-flavio-bolsonaro.shtml
getting page https://www1.folha.uol.com.br/poder/2019/02/senador-propoe-convocar-ministro-do-turismo-para-falar-de-esquema-de-laranjas.shtml
getting page https://www1.folha.uol.

getting page https://www1.folha.uol.com.br/poder/2018/12/operador-de-cabral-aponta-19-corruptores-ainda-nao-denunciados.shtml
getting page https://www1.folha.uol.com.br/poder/2018/11/frente-parlamentar-quer-dividir-com-novo-governo-pauta-anticorrupcao.shtml
getting page https://www1.folha.uol.com.br/poder/2018/12/opostos-politicamente-bolsonaro-e-lula-tem-aspectos-em-comum.shtml
getting page https://www1.folha.uol.com.br/poder/2018/12/gsi-recomenda-cautela-a-equipe-de-bolsonaro-em-cerimonia-de-posse.shtml
getting page https://www1.folha.uol.com.br/poder/2018/12/paulo-guedes-cancela-ida-a-europa-por-motivos-de-saude.shtml
getting page https://www1.folha.uol.com.br/poder/2019/02/onyx-atribui-crise-com-filho-de-bolsonaro-e-ministro-a-pouco-tempo-de-governo.shtml
getting page https://www1.folha.uol.com.br/poder/2019/02/bebianno-diz-que-nao-pretende-pedir-demissao-e-que-decisao-cabe-a-bolsonaro.shtml
getting page https://www1.folha.uol.com.br/poder/2019/02/ministro-de-bolsonaro-criou-candid